In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import datetime as dt
import numpy as np
import scipy as sp
import pandas as pd
import geopandas as gpd
import intake,param
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

from tqdm import tqdm, trange
import ipywidgets as iw

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr
import xarray.ufuncs as xu

from holoviews import opts
from holoviews.util import Dynamic
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize

from holoviews.streams import *
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

import panel as pn

import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf


hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

# set pandas dataframe float precision 
pd.set_option('display.precision',2)


In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")    

In [ ]:
import utils as u
import hv_utils as  hvu


In [ ]:
mro = u.get_mro

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

## Set default holoviews style options

In [ ]:
%opts Image [colorbar=True, tools=['hover'], active_tools=['wheel_zoom']] Curve [tools=['hover']]

In [ ]:
opts.defaults(
    opts.WMTS(active_tools=['wheel_zoom']),
    opts.Image(active_tools=['wheel_zoom'], tools=['hover'], colorbar=True),
    opts.Curve(active_tools=['wheel_zoom'], tools=['hover']),
    opts.Scatter(active_tools=['wheel_zoom'], tools=['hover']),
    opts.HLine(active_tools=['wheel_zoom'], tools=['hover']),

    opts.RGB(active_tools=['wheel_zoom']),
    opts.Overlay(active_tools=['wheel_zoom']),
)


In [ ]:
H,W = 500,500

---
## Load Datasets

In [ ]:
# Southern Africa Dataset
data_dir = Path.home()/'data'
fpath_sa = str(
    data_dir/'mint/FLDAS/FLDAS_NOAH01_A_SA_D.001/2019/04/FLDAS_NOAH01_A_SA_D.A201904*.001.nc'
)
fpath_ea = str(
    data_dir/'mint/FLDAS/FLDAS_NOAH01_A_EA_D.001/2019/04/FLDAS_NOAH01_A_EA_D.A201904*.001.nc'
)
ds_sa = xr.open_mfdataset(fpath_sa)
ds_sa = ds_sa.drop_dims('bnds')

ds_ea = xr.open_mfdataset(fpath_ea)
ds_ea = ds_ea.drop_dims('bnds')

         
# print(ds_ea)
# print(ds_sa)

In [ ]:
xrd_ea = ds_ea.persist()
xrd_sa = ds_sa.persist()

In [ ]:
# data variable list
varnames_ea = list(ds_ea.data_vars.keys())
varnames_sa = list(ds_sa.data_vars.keys())
varnames = varnames_ea
varname = varnames[3]
print(varname)

# create holoviews dataset containers 
kdims = ['X','Y','time']
hvd_ea = hv.Dataset(xrd_ea, kdims)
hvd_sa = hv.Dataset(xrd_sa, kdims)


In [ ]:
# colormaps
## discretize it conveniently using holoview's "color_level" option
t_fixed = '2019-04-05'
varname = varnames[5] 
print("Selecting a datavariable at a fixed time point: ", t_fixed, varname)

# timg_ea = hvd_ea.select(time=t_fixed).to(gv.Image, kdims=['X', 'Y'], vdims=varname) #this returns a holomap, not a hv.Image object
# To construct an hv.Image object, we need to pass in the xr.DataArray (ie. one value variable)
print(xrd_ea[varname].isel(time=3) )
timg_ea = gv.Image(xrd_ea[varname].isel(time=3) , ['X','Y'], crs=ccrs.PlateCarree()) #Opt: vdims=varname
timg_sa = gv.Image(xrd_sa[varname].isel(time=3) , ['X','Y'], crs=ccrs.PlateCarree()) #Opt: vdims=varname
# print(timg_sa)
# gv.tile_sources.Wikipedia * timg_ea.opts(alpha=0.5,width=W_IMG, height=H_IMG) #+ timg_sa.opts(width=W_IMG, height=H_IMG)

## Basemap tile

We need to handle the projection from latlon to web mercator (which is what the hv.tiles expect).


In [ ]:
wmts_url = 'https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png'
basemap = gv.tile_sources.EsriImagery

---
## Back to FLDAS
Modified: Jun 25, 2019

Combining holoviews objects with Bokeh models for custome interactive visualization


In [ ]:
# Set extra style opts (in addition to default from above)
W_IMG = 500; H_IMG = 500
W_PLOT = 300; H_PLOT = 300

In [ ]:
scatter_opts = dict(width=W_PLOT, height=H_PLOT,
                    tools=['hover', 'tap'], 
                    framewise = True)
curve_opts = dict(width=W_PLOT, height=H_PLOT,
                  framewise=True)
img_opts = dict(width=W_IMG, height=H_IMG,
                axiswise=True, 
                framewise=False,
                tools=['hover', 'tap'],
                colorbar=True
               )
wmts_opts = dict(width=W_IMG, height=H_IMG)

tbl_opts = dict(width = W_PLOT)

# datashader opts
ds_opts = dict(width=W_IMG, height=H_IMG,
#             x_sampling=0.5, 
#             y_sampling=0.5,
            )

## Put the holoviews objects together 
- to form a parameterized class object

Modified: Jun 27, 2019


In [ ]:
trange = list(map(pd.Timestamp, hvd_ea.range('time')))

class ZonalExplorer(param.Parameterized):
    
    ################################################################################
    ## Parameters
    ################################################################################
    region = param.ObjectSelector(default='EA', objects=['EA', 'SA'])
    varname = param.ObjectSelector(default=varnames[10], objects=varnames)
    
    
    ################################################################################
    ## Initialize instance
    ################################################################################
    def __init__(self, **params):
        super().__init__(**params)
        self._update()
        
#         self._init_streams()
#         self._set_dyn_main()
#         self._set_dyn_funcs()
        
    @param.depends('region', 'varname', watch=True)
    def _update(self):
        """
        (Re)Set main attributes: region data, time_values, empty_tplot
        in reaction to region and variable name parameter change from the user
        """
        print("Update is called")
        self.xrd = xrd_ea if self.region == 'EA' else xrd_sa
        self.time_values = u.to_datetime(self.xrd.get_index('time'))
        self.empty_tplot = self.get_empty_tplot()
        self.vlines = self.get_dmap_vlines()
        self.dmap_timg = self.get_dmap_timg()
        self.realize_dmaps()
        
        # add polygons and reset boxedit stream
        self.polys = gv.Polygons([], crs=ccrs.PlateCarree())
        self.box_stream = BoxEdit(source=self.polys)
        self.dmap_timg = basemap * self.dmap_timg * self.polys
        self.dmap_roi_curves = hv.DynamicMap(self.roi_curves, streams=[self.box_stream])
        print("Finished updating xrd, time_values, empty_tplot, dmap_vlines)")
        
    def _set_streams(self):
        # Set BoxEdit stream
        self.polys = gv.Polygons([], crs=ccrs.PlateCarree())
        self.box_stream = BoxEdit(source=self.polys)

    ################################################################################
    ## Basic Building Blocks -- generic functions
    ################################################################################  
    def get_empty_tplot(self):
        """
        Get an 'empty' time curve (hv.Curve) with the xaxis set as dt.datetime values
        correpsonding to self.time_values. This can serve as the background plot for 
        hv.VLine and roi_curves for boxedit selection functionality
        
        Returns:
        - hv.Curve
        """
        print("get_empty_tplot is called")
        dummy_df = pd.DataFrame({'time': self.time_values, 
                                 self.varname: np.zeros(len(self.time_values))})
        empty_tplot= hv.Curve(dummy_df, 'time', self.varname).opts(line_alpha=0., 
                                                                  bgcolor=(0,0,0,0))
        return empty_tplot
    
    def get_dmap_vlines(self, **vline_opts):
        """
        Construct a hv.Holomap of VLines at each time point in self.time_values
        
        Returns:
        hv.Holomap of hv.VLines with kdims of time
        """
        vlines = hv.DynamicMap(lambda time: self.empty_tplot * hv.VLine(time), kdims='time') 
        return vlines#.opts(**vline_opts)
        
    def get_dmap_timg(self):
        dmap_timg = datashade(hv.DynamicMap(self.get_timg, kdims='time'),
                      **ds_opts)
        return dmap_timg

    def get_timg(self, time):
        print("Getting a new timg...")
        print(f'-- Time: {time}, Region: {self.region}, Variable: {self.varname}')
        
        data = self.xrd[self.varname].sel(time=time)
        return gv.Image(data, ['X','Y'], self.varname, crs=ccrs.PlateCarree())
    
    def realize_dmaps(self):
        """
        Set the values for dimensions of dynamic maps
        Inplace operation on any dynamic map type attributes
        """
        self.vlines = self.vlines.redim.values(time=self.time_values)
        self.dmap_timg = self.dmap_timg.redim.values(time=self.time_values)

        
    ################################################################################
    ## Link streams Callbacks
    ################################################################################
    def roi_curves(self, data):
        if not data or not any(len(d) for d in data.values()):
            return hv.NdOverlay({0: self.empty_tplot})
        curves = {}
        data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
        for i, (x0,x1,y0,y1) in enumerate(data):
            selection = hv.Dataset( self.xrd[self.varname].sel(X=slice(x0,x1), Y=slice(y0,y1)), 
                                   kdims=['X','Y','time'])
            curves[i] = hv.Curve(selection.aggregate('time', np.nanmean), 'time', self.varname)
        return hv.NdOverlay(curves, label='roi_curves').opts(
            opts.Curve(framewise=True))
    
    
    ################################################################################
    ## Print states
    ################################################################################ 
    def pp(self):
        self.print_param_values()
        
    ################################################################################
    ## Layouts and Views
    ################################################################################
    @param.depends('region', 'varname', watch=True)
    def panel_roi(self):
        layout = (self.dmap_timg + self.dmap_roi_curves*self.vlines).opts(
                opts.Curve(width=400,framewise=True),
                opts.VLine(color='black'),
                opts.Polygons(fill_alpha=0.2, line_color='white', fill_color='orange'), 
                )
        p = pn.panel(layout)
        roi_twidget =  p[1][0]
        roi_main = p[0]
        
        # Customize the components and final arrangement
        
        return pn.Column(roi_main, roi_twidget)

In [ ]:
ze = ZonalExplorer()

In [ ]:
## initialization test
# ex.pp()

In [ ]:
# ex.vlines
# ex.empty_tplot
# ex.dmap_timg + ex.vlines

In [ ]:
pn.Column(
    pn.panel(ze.param),
    pn.panel(ze.panel_roi)
).servable()